In [3]:
# importing libraries

import pandas as pd
import tweepy
import json
import datetime
import re
import GetOldTweets3 as got
import requests

# importing libraries for geo-locations
from googlemaps import places

In [4]:
pwd

'C:\\Users\\Wilson\\Documents\\capstone\\brewery_recommender\\data_collection'

In [5]:
# create a json file with your twitter keys and read it in here for privacy
with open('../../twitter_keys.json') as f:
    keys = json.load(f)

In [8]:
# setting twitter keys from json file
key = keys['key']
key_secret = keys['key_secret']
token = keys['token']
token_secret = keys['token_secret']

In [9]:
# Authenticate Twitter API using  account Keys 
auth_details = tweepy.OAuthHandler(key, key_secret)
auth_details.set_access_token(token, token_secret)
my_accounts = []

# create api instance
api = tweepy.API(auth_details, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [10]:
# check username and api keys
user = api.me()
print(user.name)
print(api.rate_limit_status()['resources']['search'])

gswilson6
{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1573684672}}


In [26]:
# reading in the googlemaps_api key from an external file
with open ("../../googlemaps_api_key.txt", "r") as myfile:
    api_key = myfile.read().replace('\n', '')

In [27]:
# function to find the location of the user based on a string query passed into googlemaps_api

def places_search(place):
    places = [] 
    for i in range(len(place.split())):
        if i < (len(place.split())-1):
            places.append(place.split()[i]+'%20')
        else:
            places.append(place.split()[i])
    place_new = ''.join(places)
    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place_new}&inputtype=textquery&fields=geometry&key='+api_key
    response = requests.get(url)
    resp_json_payload = response.json()
    lat = str(resp_json_payload['candidates'][0]['geometry']['location']['lat'])
    lng = str(resp_json_payload['candidates'][0]['geometry']['location']['lng'])
    return lat+','+lng

In [28]:
# function to find beer tweets within a defined radius of the user and returning the tweets
# in csv format
def beer_tweets_near_me(place, radius):
    coords = places_search(place)
    snake_places = []
    for i in place.split():
        snake_places.append(i+'_')
    snake_place = ''.join(snake_places)
    text = []
    user = []
    date = []
    beer_tweet = {}
    n_tweets = 1_000
    for i in tweepy.Cursor(api.search, q='brewery', geocode=coords+','+str(radius)+'km').items(n_tweets):
        text.append(i.text)
        user.append(i.author.screen_name)
        date.append(i.created_at)
    beer_tweet['tweet'] = text
    beer_tweet['user'] = user
    beer_tweet['date'] = date
    beer_tweets = pd.DataFrame(beer_tweet)
    return beer_tweets.to_csv(f'../data/{snake_place}beer_tweets_{radius}_km.csv', index=False)

In [29]:
# a list of dsi east coast campuses to loop through
DSI_ec_locations = ['Atlanta', 'Austin', 'Boston', 'Chicago', 'DC', 'New York']

In [30]:
# creating csv's for each dsi east coast location
for location in DSI_ec_locations:
    beer_tweets_near_me('General Assembly '+location, 10)

In [13]:
beer_tweets_near_me('Kalamazoo', 1000)